<a href="https://colab.research.google.com/github/imadegunawinangun/PrediksiSuhuDenpasar/blob/main/Prediksi_Suhu_Denpasar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import random
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from scipy import stats


from tensorflow import keras
from tensorflow.keras import layers


In [2]:
!git clone https://github.com/imadegunawinangun/PrediksiSuhuDenpasar

Cloning into 'PrediksiSuhuDenpasar'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 1), reused 7 (delta 1), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [3]:
ls PrediksiSuhuDenpasar/

openweatherdata-denpasar-1990-2020.csv
openweatherdata-denpasar-1990-2020v0.1.csv


In [4]:
data = pd.read_csv('PrediksiSuhuDenpasar/openweatherdata-denpasar-1990-2020.csv')
data.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,rain_6h,rain_12h,rain_24h,rain_today,snow_1h,snow_3h,snow_6h,snow_12h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,631152000,1990-01-01 00:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,25.82,25.82,25.82,1010.0,NaN,NaN,86,1.36,225,0.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98,500,Rain,light rain,10d
1,631155600,1990-01-01 01:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,26.20,26.20,26.20,1011.0,NaN,NaN,84,2.09,247,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91,500,Rain,light rain,10d
2,631159200,1990-01-01 02:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,26.45,26.45,26.45,1011.0,NaN,NaN,84,2.44,262,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94,500,Rain,light rain,10d
3,631162800,1990-01-01 03:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,26.80,26.80,26.80,1011.0,NaN,NaN,82,2.29,271,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94,500,Rain,light rain,10d
4,631166400,1990-01-01 04:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,27.04,27.04,27.04,1010.0,NaN,NaN,82,1.71,274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,803,Clouds,broken clouds,04d


In [5]:
data.tail()

,dt,dt_iso,timezone,city_name,lat,lon,temp,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,rain_6h,rain_12h,rain_24h,rain_today,snow_1h,snow_3h,snow_6h,snow_12h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
264919,1578423600,2020-01-07 19:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,27.0,27.0,27.0,1007.0,NaN,NaN,94,4.1,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
264920,1578427200,2020-01-07 20:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,27.0,27.0,27.0,1007.0,NaN,NaN,94,5.7,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
264921,1578430800,2020-01-07 21:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,28.0,28.0,28.0,1007.0,NaN,NaN,83,6.7,290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
264922,1578434400,2020-01-07 22:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,28.0,28.0,28.0,1007.0,NaN,NaN,83,6.2,290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03n
264923,1578438000,2020-01-07 23:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,28.0,28.0,28.0,1008.0,NaN,NaN,83,5.7,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03d


In [6]:
data = pd.read_csv('PrediksiSuhuDenpasar/openweatherdata-denpasar-1990-2020.csv', 
                   low_memory=False
                   )

data.drop(["dt", "timezone", 'city_name', 'lat', 'lon', 'sea_level','grnd_level', 'rain_1h', 'rain_3h','rain_6h','rain_12h','rain_24h','rain_today','snow_1h','snow_3h','snow_6h','snow_12h','snow_24h','snow_today'], inplace=True, axis=1)

In [7]:
data

,dt_iso,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1990-01-01 00:00:00 +0000 UTC,25.82,25.82,25.82,1010.0,86,1.36,225,98,500,Rain,light rain,10d
1,1990-01-01 01:00:00 +0000 UTC,26.20,26.20,26.20,1011.0,84,2.09,247,91,500,Rain,light rain,10d
2,1990-01-01 02:00:00 +0000 UTC,26.45,26.45,26.45,1011.0,84,2.44,262,94,500,Rain,light rain,10d
3,1990-01-01 03:00:00 +0000 UTC,26.80,26.80,26.80,1011.0,82,2.29,271,94,500,Rain,light rain,10d
4,1990-01-01 04:00:00 +0000 UTC,27.04,27.04,27.04,1010.0,82,1.71,274,76,803,Clouds,broken clouds,04d
...,...,...,...,...,...,...,...,...,...,...,...,...,...
264919,2020-01-07 19:00:00 +0000 UTC,27.00,27.00,27.00,1007.0,94,4.10,300,75,803,Clouds,broken clouds,04n
264920,2020-01-07 20:00:00 +0000 UTC,27.00,27.00,27.00,1007.0,94,5.70,300,75,803,Clouds,broken clouds,04n
264921,2020-01-07 21:00:00 +0000 UTC,28.00,28.00,28.00,1007.0,83,6.70,290,75,803,Clouds,broken clouds,04n
264922,2020-01-07 22:00:00 +0000 UTC,28.00,28.00,28.00,1007.0,83,6.20,290,40,802,Clouds,scattered clouds,03n


In [8]:
data.isnull().sum()


dt_iso                 0
temp                   0
temp_min               0
temp_max               0
pressure               0
humidity               0
wind_speed             0
wind_deg               0
clouds_all             0
weather_id             0
weather_main           0
weather_description    0
weather_icon           0
dtype: int64

In [9]:
dates = data['dt_iso'].values
temp  = data['temp'].values
 
 
plt.figure(figsize=(15,5))
plt.plot(dates, temp)
plt.title('Temperature average',
          fontsize=20);

Error in callback <function flush_figures at 0x7f59af2f24d0> (for post_execute):


KeyboardInterrupt: ignored

In [10]:
data.drop(['pressure','humidity','wind_speed','wind_deg','clouds_all', 'weather_id', 'weather_main', 'weather_description', 'weather_icon'], inplace=True, axis=1)

In [11]:
data

,dt_iso,temp,temp_min,temp_max
0,1990-01-01 00:00:00 +0000 UTC,25.82,25.82,25.82
1,1990-01-01 01:00:00 +0000 UTC,26.20,26.20,26.20
2,1990-01-01 02:00:00 +0000 UTC,26.45,26.45,26.45
3,1990-01-01 03:00:00 +0000 UTC,26.80,26.80,26.80
4,1990-01-01 04:00:00 +0000 UTC,27.04,27.04,27.04
...,...,...,...,...
264919,2020-01-07 19:00:00 +0000 UTC,27.00,27.00,27.00
264920,2020-01-07 20:00:00 +0000 UTC,27.00,27.00,27.00
264921,2020-01-07 21:00:00 +0000 UTC,28.00,28.00,28.00
264922,2020-01-07 22:00:00 +0000 UTC,28.00,28.00,28.00


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
data=scaler.fit_transform(data)

ValueError: ignored

In [ ]:
train = int(len(data)*0.8)
test = len(data)-train

In [ ]:
train

In [ ]:
test

In [ ]:
train_data, valid_data = data[0:train,:],data[train:len(data),:1]